In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
# Optimisation parameters
problem_name = "Forrester"
problem_params = {}
time_name = "negcorrtime"
n_workers = 2
acq_name = "HardLocalPenalisationBatchBOCost"
bo_name = "AsyncSKBO"
kill_name = "DeterministicKilling"
#kill_name = None
run_no = 1
budget = 1000
verbose = False
q=1

acq_params = {
    "n_opt_samples": 1 * 1000,
    "n_opt_bfgs": 10,
    "acq_name": "EI"
}

killing_params = {
    "delta": 0,
    "acq_name": "HardLocalPenalisationBatchBOCost",
    "acq_params": acq_params,
    "n_opt_samples": 1 * 1000,
    "n_opt_bfgs": 10,
}

# Left as defaults
save_every = 10
repeat_no = None

In [3]:
import torch
import os
from aegis import test_problems, time_dists, batch, optim, util
from aegis.gen_training_data import generate_training_data_LHS

generate_training_data_LHS(problem_name, n_exp_start=1, n_exp_end=1)

# set up the saving paths
save_path = util.generate_save_filename(
    time_name,
    problem_name,
    budget,
    n_workers,
    acq_name,
    run_no,
    bo_name,
    problem_params,
    acq_params,
    repeat_no=repeat_no,
)

if os.path.exists(save_path):
    load_path = save_path
    print("Loading saved run")
else:
    load_path = util.generate_data_filename(
        problem_name, run_no, problem_params, repeat_no=repeat_no
    )

# load the training data
data = torch.load(load_path)
Xtr = data["Xtr"]
Ytr = data["Ytr"]

# if it has additional arguments add them to the dictionary passed to f
if "problem_params" in data:
    problem_params.update(data["problem_params"])

print(f"Training data shape: {Xtr.shape}")

# load the problem instance
f = getattr(test_problems, problem_name)(**problem_params)

# wrap the problem for torch and so that it resides in [0, 1]^d
f = util.TorchProblem(util.UniformProblem(f))

# instantiate the time function
time_class = getattr(time_dists, time_name)

# get the acquisition function class
acq_class = getattr(batch, acq_name)

# get the BO class
asbo = getattr(optim, bo_name)

# run the BO
if kill_name is not None:
    asbo = asbo(
        f,
        Xtr,
        Ytr,
        acq_class,
        acq_params,
        budget,
        n_workers,
        time_class,
        q=q,
        verbose=verbose,
        kill_name=kill_name,
        killing_params=killing_params
    )
else:
    asbo = asbo(
        f,
        Xtr,
        Ytr,
        acq_class,
        acq_params,
        budget,
        n_workers,
        time_class,
        q=q,
        verbose=verbose,
    )

/home/lewis/anaconda3/envs/mp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


File exists, skipping: data/Forrester_001.pt
AsyncSKBO
Training data shape: torch.Size([2, 1])


In [4]:
import matplotlib.pyplot as plt
import plotting
import numpy as np
from aegis.batch import ratios, penalisation

acq_params = {
    "n_opt_samples": 1 * 1000,
    "n_opt_bfgs": 10,
}

asbo.step()

res = asbo.get_results()
models = asbo.get_models()

T_data = asbo.output_transform(asbo.Ytr)
T_time = asbo.output_transform(asbo.time_taken)

fig, axs = plt.subplots(4, figsize=(7,9))

color = np.array([31, 119, 180])/255
plotting.gp_plot(axs[0], models["ProblemModel"], T_data, color=color)
axs[0].set(title="Problem Model")
if "CostModel" in models:
    plotting.gp_plot(axs[1], models["CostModel"], T_time, color=color)
    axs[1].set(title="Cost Model")

# Get EI for comparison
EI = ratios.EI(
    models["ProblemModel"],
    asbo.f.lb,
    asbo.f.ub,
    under_evaluation=asbo.ue.get(),
    **acq_params,
)
plotting.acq_plot(axs[2], EI)
axs[2].set(title="EI For Comparison")

# Acq function we're using
plotting.acq_plot(axs[3], asbo.acq)
axs[3].set(title="Acqisition Function")

# Mark our choice
plotting.mark_acq_choice_plot(axs, asbo.acq, color='red')

# Mark ongoings
for x in asbo.ue.get():
    plotting.plot_vline_axes(axs, x, color="blue")

# Mark EI's choice for comparison
#plotting.mark_acq_choice_plot(axs, EI, color='green')

fig.tight_layout()

/home/lewis/anaconda3/envs/mp/lib/python3.10/site-packages/gpytorch/lazy/triangular_lazy_tensor.py:130: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:1672.)
  res = torch.triangular_solve(right_tensor, self.evaluate(), upper=self.upper).solution


%%%%%%%%%%%%%%%%%%%
x_star: 0.14558493621844829, val:0.017431421488805145
x_i: 0.010599449630607388, val:0.0
%%%%%%%%%%%%%%%%%%%
here: tensor([[0.0106]], dtype=torch.float64)
Returning: tensor([[0.0106]], dtype=torch.float64)
Killing tensor([[0.0106]], dtype=torch.float64)
%%%%%%%%%%%%%%%%%%%
x_star: 0.0, val:0.016474078486654206
x_i: 0.14558493621844829, val:0.0
%%%%%%%%%%%%%%%%%%%
here: tensor([[0.1456]], dtype=torch.float64)
Returning: tensor([[0.1456]], dtype=torch.float64)
Killing tensor([[0.1456]], dtype=torch.float64)
%%%%%%%%%%%%%%%%%%%
x_star: 0.14669890210707595, val:0.016933867185292474
x_i: 0.0, val:0.0
%%%%%%%%%%%%%%%%%%%


KeyboardInterrupt: 